In [4]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Load the "fer2013" dataset
data = pd.read_csv('D:/face/train.csv')

# Preprocess the data
images = []
for pixels in data['pixels']:
    img = np.array([int(pixel) for pixel in pixels.split(' ')], dtype=np.uint8).reshape((48, 48))
    images.append(img)

X = np.array(images)
y = data['emotion']

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 classes for 7 emotions (anger, disgust, fear, happy, sad, surprise, neutral)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the data to fit the model input
X_train_reshaped = X_train.reshape(-1, 48, 48, 1)
X_test_reshaped = X_test.reshape(-1, 48, 48, 1)

# Train the model
model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test))


Epoch 1/20
718/718 [==============================] - 21s 29ms/step - loss: 1.7028 - accuracy: 0.3155 - val_loss: 1.5148 - val_accuracy: 0.4248
Epoch 2/20
718/718 [==============================] - 24s 33ms/step - loss: 1.4492 - accuracy: 0.4437 - val_loss: 1.3880 - val_accuracy: 0.4688
Epoch 3/20
718/718 [==============================] - 28s 39ms/step - loss: 1.3218 - accuracy: 0.4955 - val_loss: 1.2960 - val_accuracy: 0.5094
Epoch 4/20
718/718 [==============================] - 27s 37ms/step - loss: 1.2421 - accuracy: 0.5268 - val_loss: 1.2807 - val_accuracy: 0.5148
Epoch 5/20
718/718 [==============================] - 26s 36ms/step - loss: 1.1819 - accuracy: 0.5536 - val_loss: 1.2575 - val_accuracy: 0.5294
Epoch 6/20
718/718 [==============================] - 26s 36ms/step - loss: 1.1237 - accuracy: 0.5777 - val_loss: 1.2352 - val_accuracy: 0.5340
Epoch 7/20
718/718 [==============================] - 26s 36ms/step - loss: 1.0705 - accuracy: 0.5993 - val_loss: 1.2416 - val_accuracy:

In [8]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test Accuracy: {test_accuracy}")

180/180 [==============================] - 2s 11ms/step - loss: 1.9187 - accuracy: 0.5251
Test Accuracy: 0.5250783562660217


In [9]:
# Save the trained model to an HDF5 file
model.save('emotion_detection_model1.h5')

C:\Users\adity\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
import cv2
import pandas as pd
import time
import pygame

video_url = 'http://192.168.2.117:4747/video'
cap = cv2.VideoCapture(video_url)

# Check if the video stream was opened successfully
if not cap.isOpened():
    print("Error opening video stream.")
    exit()

import tensorflow as tf

# Load the pre-trained emotion detection model
emotion_model = tf.keras.models.load_model('C:/Users/adity/Downloads/emotion_detection_model1.h5')

# Define the emotion labels (same as in the previous code)
emotion_labels = ['anger', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load the music features from the CSV file
music_features_data = pd.read_csv('D:/face/Data/features_3_sec.csv')

# Extract the relevant features for music recommendation (same as the emotion detection features)
X_music_features = music_features_data.iloc[:, 2:-1].values

# Normalize the music features
# Normalize pixel values to [0, 1]
X_music_features = X_music_features / 255.0

# Create a list to store recommended songs for each emotion
song_recommendations = {
    'anger': ['rock', 'rock', 'rock'],
    'disgust': ['hiphop', 'hiphop', 'hiphop'],
    'fear': ['classical', 'classical', 'classical'],
    'happy': ['hiphop', 'hiphop', 'hiphop'],
    'sad': ['blues', 'blues', 'blues'],
    'surprise': ['hiphop', 'hiphop', 'hiphop'],
    'neutral': ['blues', 'blues', 'blues']
}

# Define the path where songs are stored
songs_path = 'D:/face/Data/genres_original/'

# Initialize pygame mixer
pygame.mixer.init()

prev_time = time.time()

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Calculate the time elapsed since the last prediction
    current_time = time.time()
    time_elapsed = current_time - prev_time

    # Perform emotion prediction and recommend a song after every 5 seconds
    if time_elapsed >= 5.0:
        prev_time = current_time

        # Preprocess the frame (convert to grayscale, resize to 48x48, normalize)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (48, 48))
        normalized_frame = resized_frame / 255.0
        input_frame = normalized_frame.reshape(-1, 48, 48, 1)

        # Perform emotion detection on the frame
        predicted_emotions = emotion_model.predict(input_frame)
        predicted_emotion_class = predicted_emotions.argmax(axis=1)[0]
        predicted_emotion = emotion_labels[predicted_emotion_class]

        # Get song recommendations for the predicted emotion
        recommended_songs = song_recommendations[predicted_emotion]

        # Print the emotion label and recommended songs
        print(f"Emotion: {predicted_emotion}")
        print("Recommended songs:")
        for song in recommended_songs:
            print(song)
        print("\n")

        # Play the first recommended song from the provided path
        if recommended_songs:
            song_path = songs_path + recommended_songs[0] +'/'+ recommended_songs[0] + '.00000.wav'
            pygame.mixer.music.load(song_path)
            pygame.mixer.music.play()

    # Show the frame with the emotion label
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video stream and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 60ms/step
Emotion: happy
Recommended songs:
hiphop
hiphop
hiphop


1/1 [==============================] - 0s 16ms/step
Emotion: fear
Recommended songs:
classical
classical
classical


1/1 [==============================] - 0s 17ms/step
Emotion: happy
Recommended songs:
hiphop
hiphop
hiphop


1/1 [==============================] - 0s 20ms/step
Emotion: neutral
Recommended songs:
blues
blues
blues


1/1 [==============================] - 0s 19ms/step
Emotion: sad
Recommended songs:
blues
blues
blues


